#### Results Preparation for Plotting


In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import rasterio as rio
from mpl_toolkits.basemap import Basemap  
import matplotlib.pyplot as plt
from shapely.ops import unary_union   
import os
import glob
from geopandas.tools import overlay


##### Make Data Ready

In [2]:
# Append all gdf for easy management
# Create empty GeoDataFrames for each type
all_DEM_labels_results = gpd.GeoDataFrame()
all_OP_labels_results = gpd.GeoDataFrame()
all_OT_labels_results = gpd.GeoDataFrame()

types = ['DEM', 'OP', 'OT']
regions = ['herschel', 'peel', 'kolguev', 'gydan']

for type_ in types:
    for region in regions:
        # Read the data
        data = gpd.read_file(f'Results\\all\\{region}_{type_}_labels_results.geojson')

        # Append the data to the corresponding GeoDataFrame
        if type_ == 'DEM':
            all_DEM_labels_results = pd.concat([all_DEM_labels_results, data], ignore_index=True)
        elif type_ == 'OP':
            all_OP_labels_results = pd.concat([all_OP_labels_results, data], ignore_index=True)
        elif type_ == 'OT':
            all_OT_labels_results = pd.concat([all_OT_labels_results, data], ignore_index=True)

all_OP_labels_results.drop(columns=['level_0', 'level_1'], inplace=True)
all_OT_labels_results.drop(columns=['level_0', 'level_1'], inplace=True)

In [3]:
# Create gdf for all DEM and all Optical Training which are in overlapping study area
all_fps = gpd.GeoDataFrame()

for region in regions:
    files = glob.glob(f'Results\\{region}\\Footprints\\footprint_{region}_*.geojson')

    for file in files:
        # Extract region and year from the filename
        filename = os.path.basename(file)
        year = filename.split('_')[-1].split('.')[0][-4:]
        
        fp = gpd.read_file(file)
        
        # Add columns for region and year
        fp['Region'] = region
        fp['Year'] = year
        
        all_fps = pd.concat([all_fps, fp], ignore_index=True)

# Create a single footprint
fps_union = unary_union(all_fps.geometry)

# Save the resulting footprints
all_fps.to_file('Plotting\\boundries\\all_footprints.geojson', driver='GeoJSON')

# Convert DataFrame to GeoDataFrame and set CRS
all_DEM_labels_results_gdf = gpd.GeoDataFrame(all_DEM_labels_results, geometry='geometry', crs='EPSG:3413')

# Perform overlay operation to retain attributes
intersected_dem_labels = overlay(all_DEM_labels_results_gdf, gpd.GeoDataFrame(geometry=[fps_union], crs='EPSG:3413'), how='intersection')

# Save the resulting intersected DEM labels
intersected_dem_labels.to_file('Plotting\\labels\\common_DEM_labels.geojson', driver='GeoJSON')


In [4]:
# Save the appended GeoDataFrames for easy comparison
all_DEM_labels_results.to_file('Plotting\\labels\\all_DEM_labels.geojson', driver='GeoJSON')
all_OP_labels_results.to_file('Plotting\\labels\\all_OP_labels.geojson', driver='GeoJSON')

intersected_dem_labels.to_file('Plotting\\labels\\common_DEM_labels.geojson', driver='GeoJSON')
all_OT_labels_results.to_file('Plotting\\labels\\all_OT_labels.geojson', driver='GeoJSON')

In [5]:
# Create Common labelling area boundries
all_areas = gpd.read_file('Plotting\\boundries\\all_areas.geojson')
all_footprints = gpd.read_file('Plotting\\boundries\\all_footprints.geojson')

common_labelling_area = overlay(all_areas, all_footprints, how='intersection')
common_labelling_area.to_file('Plotting\\boundries\\common_labelling_area.geojson', driver='GeoJSON')

  ___________________________________________________________________________________________________________________________________________